In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import io
import time


In [3]:
## Scraping function - getting data from the retrieved page

def scrapeThePage(soup):  

    lst = []
    df = pd.DataFrame()
    i = 0 

    for tag in soup.find_all(class_='review-listing'):
        title = tag.find(class_='title').string
        review_ID = (tag.attrs['data-review-id'])
        link = tag.attrs['href']
        excerpt = tag.find(class_='excerpt').string
        price = tag.find(class_='price').string
        appellation = tag.find(class_='appellation').string
        #if badge != None: tag.find(class_='badge').string 
        rating = tag.find(class_='rating').text
        
        df=df.append(pd.Series([title, review_ID, link, excerpt, price, appellation, rating]), ignore_index=True)
        i=i+1

    return df   
    

In [4]:
## Function to access data

def cooknextwinesoup(startingpage, endingpage):
    
    df = pd.DataFrame()
    
    baseurl = "https://www.winemag.com/?s=&drink_type=wine&page="
    
    ## Setting up the webdriver
    option = webdriver.ChromeOptions()
    
    ## Switching off pictures to speedup the process
    chrome_prefs = {}
    option.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2} 
    chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}
    option.add_argument('--headless')
        
    chromedriver = "/Users/paskin/documents/chromedriver" # path to the chromedriver executable
    os.environ["webdriver.chrome.driver"] = chromedriver
      
    
    for i in range(startingpage,endingpage,1):
        
        time1 = time.time()
        
        workurl = baseurl + str(i)
        print(workurl)
        driver = webdriver.Chrome(chromedriver, options=option)  
        driver.get(workurl)
        
        time2 = time.time()
        print("Getting driver ready ", "--- %s seconds ---" % (time2 - time1))
        
        soup = BeautifulSoup(driver.page_source)
        driver.close()
        driver.quit()
        
        time3 = time.time()
        print("Souping ","--- %s seconds ---" % (time3 - time2))
        
        if i == startingpage:
            df = scrapeThePage(soup)
            df['page_num'] = i
        else:
            time3 = time.time()
            res = scrapeThePage(soup)
            res['page_num'] = i
            df = df.append(res)
            time4 = time.time()
            print("Appending data to df ","--- %s seconds ---" % (time4 - time3))
            
        if i % 50 ==0:
            
            df.to_csv(("Wines"+str(i)+".csv"), sep=',')
            print('Written to '+"Wines"+str(i)+".csv" )
            
        
        
        
        print('Total cycle time = ',"--- %s seconds ---" % (time.time() - time1))
        

        
        #webdriver.dispose()
        #driver.quit()
        
    return(df)
    
    